## 5.5 The QR Iteration for Matrices in Hessenberg Form

In [ ]:
import numpy as np

**Implementation 5.34: Transformation to Hessenberg form**

In [ ]:
def reduce_hessenberg(A):
    n, m = A.shape
    for i in range(m - 2):
        v = A[i + 1:, i].copy()
        ei = np.zeros(n - i - 1, dtype=A.dtype)
        ei[0] = 1
        v += np.sign(v[0]) * np.linalg.norm(v) * ei
        v /= np.linalg.norm(v)
        A[i + 1:, i:] -= 2 * np.outer(v, np.inner(v.T, A[i + 1:, i:].T))
        A[:, i + 1:] -= 2 * np.outer(np.inner(A[:, i + 1:], v), v)
    return None

Let us consider the matrix
$$A = \begin{pmatrix}
338 & -20 & -90 & 32 \\ -20 & 17 & 117 & 70
\\ -90 & 117 & 324 & -252 \\ 32 & 70 & -252 & 131
\end{pmatrix}.$$
with the following (approximate) eigenvalues

In [ ]:
A = np.array([[338, -20, -90, 32],
              [-20, 17, 117, 70],
              [-90, 117, 324, -252],
              [32, 70, -252, 131]], dtype=np.double)
A2 = A.copy()
eig0 = np.linalg.eig(A)[0]
print(eig0)

We can now check whether the reduction to Hessenberg form has changed the eigenvalues

In [ ]:
reduce_hessenberg(A)
eig1 = np.linalg.eig(A)[0]
print(f'Absolute difference = {np.linalg.norm(eig0 - eig1):6.4e}')

**Implementation 5.37: QR iteration for matrices in Hessenberg form**

To take advantage of the efficiency gains, we implement the QR factorization taking into account the Hessenberg form

In [ ]:
def qr_hessenberg(A):
    n, m = A.shape
    dtype = A.dtype
    Q = np.identity(n, dtype=dtype)
    
    for i in range(m - 1):
        v = A[i: i + 2, i].copy()
        ei = np.zeros(2, dtype=dtype)
        ei[0] = 1.0
        v += np.sign(A[i,i]) * np.linalg.norm(v) * ei
        v /= np.linalg.norm(v)
        A[i:i + 2, i:] -= 2 * np.outer(v, np.inner(v.T, A[i:i + 2, i:].T))
        Q[:, i:i + 2] -= 2 * np.outer(np.inner(Q[:, i: i + 2], v), v)
    return Q

This now allows us to implement the QR iteration for 

In [ ]:
def qr_iter_hessenberg(A, k):
    A = A.copy()
    reduce_hessenberg(A)
    for i in range(k):
        Q = qr_hessenberg(A)
        A[:, :] = A @ Q
    return np.diagonal(A)

#### Example 5.38 (Eigenvalue calculation with reduction and QR iteration)

We apply this to the previous example and obtain after ten steps the eigenvalue approximation

In [ ]:
A = A2.copy()
eig = np.linalg.eig(A)[0]
eig_qr = qr_iter_hessenberg(A, 10)
eig_qr = np.flip(np.sort(eig_qr))
print(eig_qr)

This gives the following relative errors

In [ ]:
print((abs(eig - eig_qr) / abs(eig)))

The largest relative error is about 1%. The errors in $\lambda_3$ and $\lambda_4$ are larger tha for $\lambda_1$ und $\lambda_2$, as $\lambda_3,\lambda_4$ are not separated as well.